In [4]:
import numpy as np
import pandas as pd
import pickle
import re, nltk, spacy, gensim
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from gensim.utils import simple_preprocess
from pprint import pprint
from sklearn.metrics.pairwise import euclidean_distances
from nltk.corpus import stopwords

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [2]:
dataset = pd.read_csv('../data/mortgage.csv')
dataset.head()

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,03/28/19,Mortgage,Conventional home mortgage,Struggling to pay mortgage,None,"In dispute of the loan # XXXX, for XXXX XXXX t...",Company believes it acted appropriately as aut...,"SELECT PORTFOLIO SERVICING, INC.",GA,300XX,None,Consent provided,Web,03/28/19,Closed with explanation,Yes,NaN,3193709
1,02/21/19,Mortgage,FHA mortgage,Struggling to pay mortgage,None,My complaint is with Carrington Mortgage Servi...,None,"CARRINGTON MORTGAGE SERVICES, LLC",GA,312XX,None,Consent provided,Web,02/21/19,Closed with explanation,Yes,NaN,3158652
2,06/11/19,Mortgage,Home equity loan or line of credit (HELOC),Trouble during payment process,None,As of XX/XX/2019 PHH Mortgage Services purchas...,None,PHH Mortgage Services Corporation,FL,347XX,None,Consent provided,Web,06/11/19,Closed with explanation,Yes,NaN,3270600
3,11/29/19,Mortgage,Conventional home mortgage,Trouble during payment process,None,We started our mortgage with Loan Depot in XX/...,Company believes complaint is the result of an...,"LD Holdings Group, LLC",NJ,None,None,Consent provided,Web,11/29/19,Closed with monetary relief,Yes,NaN,3453669
4,10/23/19,Mortgage,Conventional home mortgage,Struggling to pay mortgage,None,"XXXX XXXX XXXX XXXX XXXX XXXX XXXX XXXX, XXXX ...",None,"Community Loan Servicing, LLC (formerly known ...",CA,92021,None,Consent provided,Web,11/05/19,Closed with explanation,Yes,NaN,3415392


In [3]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32805 entries, 0 to 32804
Data columns (total 18 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Date received                 32805 non-null  object 
 1   Product                       32805 non-null  object 
 2   Sub-product                   32805 non-null  object 
 3   Issue                         32805 non-null  object 
 4   Sub-issue                     32805 non-null  object 
 5   Consumer complaint narrative  32805 non-null  object 
 6   Company public response       32805 non-null  object 
 7   Company                       32805 non-null  object 
 8   State                         32805 non-null  object 
 9   ZIP code                      32805 non-null  object 
 10  Tags                          32805 non-null  object 
 11  Consumer consent provided?    32805 non-null  object 
 12  Submitted via                 32805 non-null  object 
 13  D

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
dataset['Issue'].value_counts()

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Trouble during payment process                                                      15884
Struggling to pay mortgage                                                           7092
Applying for a mortgage or refinancing an existing mortgage                          5085
Closing on a mortgage                                                                3615
Incorrect information on your report                                                  677
Problem with a credit reporting company's investigation into an existing problem      335
Improper use of your report                                                            75
Unable to get your credit report or credit score                                       19
Credit monitoring or identity theft protection services                                16
Problem with fraud alerts or security freezes                                           7
Name: Issue, dtype: int64

In [5]:
dataset = dataset.drop(['Date received', 'Sub-issue', 'Company public response', 'State', 'ZIP code', 'Tags', 'Consumer consent provided?', 'Submitted via', 'Company response to consumer', 'Timely response?', 'Consumer disputed?', 'Complaint ID'], axis=1)

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Pre-processing

In [6]:
data = dataset['Consumer complaint narrative'].tolist()

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
# Remove emails
data = [re.sub(r'\S*@\S*\s?', '', sent) for sent in data]
# lowecase
data = [sent.lower() for sent in data] #lowecase
# Remove newline 
data = [re.sub(r'\s+', ' ', sent) for sent in data]
# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### Tokenization

In [8]:
nltk.download("punkt")

tokenized_data = []
token_data = [nltk.word_tokenize(words) for words in data]
for words in token_data:
    new_words = [word for word in words if word.isalnum()]
    tokenized_data.append(new_words)
#ef sent_to_words(sentences):
#    for sentence in sentences:
#        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
#tokenized_data = list(sent_to_words(data))

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


##### Stop words

In [9]:
#STOP WORDS
nltk.download('stopwords')
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'xxxx', 'xx', 'well', 'fargo', 'citibank', 'xxxxxx', 'xxxxxxxx'])
print(len(stop_words))
for key, value in dataset['Company'].value_counts().items():
    if value > 1000:
        word = key.lower().split(' ')
        for x in word:
            if len(x) > 1:
                stop_words.extend(x)
print(len(stop_words))

191
404


C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
tokenized_data = remove_stopwords(tokenized_data)

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
bigram = gensim.models.Phrases(tokenized_data, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[tokenized_data], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[tokenized_data[0]]])

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['dispute', 'loan', 'owner', 'property', 'ga', 'servicing', 'behalf', 'bank', 'look', 'forward', 'response', 'following', 'demands', 'complete', 'itemized', 'statement', 'date', 'loan', 'date', 'letter', 'insurance', 'expenses', 'related', 'thereto', 'related', 'way', 'loan', 'complete', 'itemized', 'statement', 'date', 'loan', 'date', 'letter', 'suspense', 'account', 'entries', 'corporate_advance', 'entries', 'related', 'way', 'loan', 'complete', 'itemized', 'statement', 'date', 'loan', 'date', 'letter', 'property', 'inspection', 'fees', 'property', 'preservation', 'fees', 'broker', 'opinion', 'fees', 'appraisal', 'fees', 'bankruptcy', 'monitoring', 'fees', 'similar', 'fees', 'expenses', 'related', 'way', 'loan', 'identify', 'provision', 'deed_trust', 'note', 'authorizes', 'charging', 'every', 'fee', 'loan', 'please', 'attach', 'copies', 'property', 'inspection', 'reports', 'appraisals', 'complete', 'itemized', 'statement', 'arrears', 'including', 'month', 'default', 'occurred', 'amou

In [13]:
bigram_mod

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


##### Pos tags

In [63]:
nltk.download('averaged_perceptron_tagger')
tagged_data = []
for token in tokenized_data:
    tagged_data.append(nltk.pos_tag(token))
#tagged = nltk.pos_tag(tokenized_data)
#singular_token = [word for word, tag in tagged if tag in ('NN', 'NNS', 'NNP', 'NNPS', 'PRP')]
tagged_data[0]

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


[('in', 'IN'),
 ('dispute', 'NN'),
 ('of', 'IN'),
 ('the', 'DT'),
 ('loan', 'NN'),
 ('xxxx', 'NNP'),
 ('for', 'IN'),
 ('xxxx', 'NNP'),
 ('xxxx', 'NNP'),
 ('the', 'DT'),
 ('owner', 'NN'),
 ('of', 'IN'),
 ('the', 'DT'),
 ('property', 'NN'),
 ('at', 'IN'),
 ('xxxx', 'NNP'),
 ('xxxx', 'NNP'),
 ('xxxx', 'NNP'),
 ('xxxx', 'NNP'),
 ('xxxx', 'NNP'),
 ('ga', 'NN'),
 ('you', 'PRP'),
 ('are', 'VBP'),
 ('servicing', 'VBG'),
 ('on', 'IN'),
 ('behalf', 'NN'),
 ('of', 'IN'),
 ('xxxx', 'NNP'),
 ('bank', 'NN'),
 ('i', 'NN'),
 ('look', 'VBP'),
 ('forward', 'RB'),
 ('to', 'TO'),
 ('your', 'PRP$'),
 ('response', 'NN'),
 ('to', 'TO'),
 ('the', 'DT'),
 ('following', 'VBG'),
 ('demands', 'NNS'),
 ('a', 'DT'),
 ('complete', 'JJ'),
 ('and', 'CC'),
 ('itemized', 'JJ'),
 ('statement', 'NN'),
 ('from', 'IN'),
 ('the', 'DT'),
 ('date', 'NN'),
 ('of', 'IN'),
 ('the', 'DT'),
 ('loan', 'NN'),
 ('to', 'TO'),
 ('the', 'DT'),
 ('date', 'NN'),
 ('of', 'IN'),
 ('this', 'DT'),
 ('letter', 'NN'),
 ('of', 'IN'),
 ('any', 'DT

In [72]:
post_tag_words = []
for tagged in tagged_data:
    data_words = [word for word, tag in tagged if tag in ('NN', 'NNS', 'NNP', 'NNPS', 'PRP')]
    post_tag_words.append(data_words)

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [76]:
def remove_values_from_list(the_list, val):
    return [value for value in the_list if value != val]
post_tag_new = []
for word in post_tag_words:
    word = remove_values_from_list(word, 'xxxx')
    post_tag_new.append(word)

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### Lemmatization

In [78]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']): #'NOUN', 'ADJ', 'VERB', 'ADV'
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
data_lemmatized = lemmatization(post_tag_new, allowed_postags=['NOUN','VERB']) #select noun and verb

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Vectorization

In [81]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10, 
                             token_pattern='[a-zA-Z0-9]{3,}',  
                             max_features=50000
                            )
data_vectorized = vectorizer.fit_transform(data_lemmatized)

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## MODEL

In [91]:
search_params = {'n_components': [10], 'learning_decay': [0.5, 0.7, 0.9]}
# Init the Model
lda = LatentDirichletAllocation(max_iter=5, learning_method='online', learning_offset=50.,random_state=0)
# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)
# Do the Grid Search
model.fit(data_vectorized)

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


GridSearchCV(estimator=LatentDirichletAllocation(learning_method='online',
                                                 learning_offset=50.0,
                                                 max_iter=5, random_state=0),
             param_grid={'learning_decay': [0.5, 0.7, 0.9],
                         'n_components': [10]})

In [92]:
# Best LDA model
best_lda_model = model.best_estimator_
# Model Parameters
print("Best Model's Params: ", model.best_params_)
# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)
# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Best Model's Params:  {'learning_decay': 0.7, 'n_components': 10}
Best Log Likelihood Score:  -2331726.1416995046
Model Perplexity:  315.526925199935


In [84]:
lda_output = best_lda_model.transform(data_vectorized)
# column names
topicnames = ["Topic" + str(i) for i in range(best_lda_model.n_components)]
# index names
docnames = ["Doc" + str(i) for i in range(len(data))]
# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)
# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return "color: {col}".format(col=color)
def make_bold(val):
    weight = 700 if val > .1 else 400
    return "font-weight: {weight}".format(weight=weight)

df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,dominant_topic
Doc0,0.010000,0.020000,0.000000,0.040000,0.400000,0.010000,0.000000,0.000000,0.300000,0.220000,4
Doc1,0.010000,0.110000,0.010000,0.010000,0.010000,0.130000,0.010000,0.010000,0.720000,0.010000,8
Doc2,0.040000,0.050000,0.000000,0.080000,0.040000,0.000000,0.000000,0.250000,0.440000,0.090000,8
Doc3,0.000000,0.100000,0.000000,0.140000,0.000000,0.000000,0.530000,0.000000,0.210000,0.000000,6
Doc4,0.000000,0.080000,0.000000,0.000000,0.180000,0.000000,0.000000,0.720000,0.000000,0.000000,7
Doc5,0.000000,0.000000,0.050000,0.000000,0.000000,0.410000,0.510000,0.000000,0.000000,0.000000,6
Doc6,0.000000,0.280000,0.000000,0.000000,0.630000,0.030000,0.000000,0.000000,0.000000,0.060000,4
Doc7,0.110000,0.000000,0.150000,0.400000,0.000000,0.000000,0.110000,0.000000,0.170000,0.050000,3
Doc8,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.700000,0.010000,0.230000,0.010000,6
Doc9,0.000000,0.860000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090000,0.040000,0.000000,1


In [85]:
df_topic_distribution = df_document_topic['dominant_topic'].value_counts().reset_index(name="Num Documents")
df_topic_distribution.columns = ['Topic Num', 'Num Documents']
df_topic_distribution

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Topic Num,Num Documents
0,8,7972
1,3,5048
2,1,4978
3,7,4705
4,6,3191
5,2,2209
6,9,1844
7,4,1636
8,0,950
9,5,272


In [86]:
def show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords
topic_keywords = show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=15)
# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,insurance,policy,homeowner,escrow,flood,company,letter,mortgage,coverage,home,premium,pennymac,account,year,property
Topic 1,loan,modification,mortgage,home,foreclosure,sale,time,year,payment,letter,attorney,document,bankruptcy,income,loss
Topic 2,company,mortgage,check,money,time,fund,day,house,month,week,home,refund,amount,people,year
Topic 3,loan,rate,mortgage,credit,appraisal,lender,closing,home,refinance,process,interest,close,application,fee,officer
Topic 4,mortgage,property,note,law,bank,complaint,document,state,court,debt,attorney,fraud,deed,consumer,trust
Topic 5,home,property,house,value,inspection,repair,damage,contractor,report,nationstar,work,family,hurricane,disaster,area
Topic 6,escrow,tax,payment,well,mortgage,taxis,account,property,amount,year,month,bill,loan,county,shortage
Topic 7,call,time,email,day,phone,information,document,letter,customer,number,response,request,loan,bank,week
Topic 8,payment,mortgage,account,bank,credit,fee,month,statement,loan,amount,time,letter,balance,day,company
Topic 9,loan,forbearance,payment,mortgage,payoff,servicer,month,plan,amount,program,option,interest,borrower,end,care


In [88]:
# Save best model
pickle.dump(model.best_estimator_, open('../new_preprocessing_models/best-n10-p315', 'wb'))
# Save grid
pickle.dump(model, open('../new_preprocessing_models/grid-n10-p315', 'wb'))

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
